In [1]:
%load_ext tclmagic

#  If the prev line doesnt work  install it using the next line in your unix terminal
#  /depot/Python/Python-3.8.0/bin/pip install -U tcl-magic
#  /depot/Python/Python-3.8.0/bin/pip install -U notebook
#  You can configure it to ~/.local/ 

In [2]:
%%tcl

set SHELLTOOL_LOC "/remote/cad-rep/msip/tools/Shelltools"
lappend auto_path "$SHELLTOOL_LOC/ddr-utils-lay/dev/lib/tcl"
lappend auto_path "/depot/tcl8.6.3/lib"

set RealBin [file dirname [file normalize [info script]] ]
set RealScript [file tail [file normalize [info script]] ]
set PROGRAM_NAME $RealScript
set LOGFILE "[pwd]/log-$PROGRAM_NAME.log"

package require Messaging 1.0
namespace import ::Messaging::*
package require Misc 1.0
namespace import ::Misc::*

package require cmdline
package require try
package require yaml

'0.3.10'

# Current AI

We are keeping everything since we are creating the opposite in base to the previous tests as in [notes_2023_02_17.ipynb](./notes_2023_02_17.ipynb)

In [3]:

# !cd /u/alvaro/mydisk/GitLab/my-personal-projects/Notes
# not so much from prev days...
# !cp -f ../experiments/tclyaml_experiment/inputs/sample.tcl sample.tcl

## tcl2yml

In [9]:
%%tcl


proc get_variables_from_file {__file} {
    source $__file
    unset __file
    return [info locals]
}

proc create_dict_from_file {__file} {

    source $__file

    set sv [get_variables_from_file $__file]
    
    foreach ssv $sv {
        if [regexp {(\S+)\{(\S+)\}} $ssv full dictname keyname] {
            viprint MEDIUM $full
            viprint MEDIUM "dictname = $dictname"
            viprint MEDIUM "keyname  = $keyname"
            foreach dict_values [set $full] {
                lappend list_values "- $dict_values"
            }
            set [set dictname]($keyname) $list_values
            unset list_values
            if {$dictname ni $sv} {lappend new_sv $dictname}
        } else {
            lappend new_sv $ssv
        }
    }

    foreach varname $new_sv {
        viprint LOW $varname
        if {[array exists $varname]} {
            foreach {ark arv} [array get $varname] {
                viprint LOW "$varname $ark --> $arv"
                dict set legal_release $varname "$ark: " "\{$arv\}"
            }
        } else {
            if {[llength [set $varname]]>1} {
                if {![regexp {(date|prune|layers|utility_name|repeater_name|tag|supply_pins)} $varname match]} {
                    foreach litvalue [set $varname] {
                        lappend varvalue "- $litvalue"  
                    }
                } else {
                    set varvalue [set $varname]
                }
            } else {
                set varvalue [set $varname]
            }
            
            dict set legal_release $varname $varvalue
            unset varname
            unset varvalue
        }
        
        
    }
    return $legal_release
}

proc save_cleaned_yml_to_file {sample_path {file_name "yml-cleaned.yml"}} {

    set legal_release [create_dict_from_file $sample_path]
    set yaml_format [yaml::dict2yaml $legal_release 4 100]

    set cleaned_format [regsub -all {\: \} \{} $yaml_format ": "]
    set cleaned_format [regsub -all {\} \{|\|-\n    \{|>\n    } $cleaned_format "\n    " ]
    
    set cleaned_values [regsub -all {\: \{} $cleaned_format ": " ]
    set cleaned_values [regsub -all {\}\n} [regsub -all {\}\n} $cleaned_values "\n" ] "\n" ]
    write_file $file_name [regsub -all {\}} [regsub -all {: \{-} $cleaned_values ":\n    -" ] "" ]
 
}
set sample "/slowfs/dcopt103/alvaro/GitLab/ddr-hbm-phy-automation-team/ddr-ckt-rel/dev/main/tests/data/test_legal_release"

save_cleaned_yml_to_file  $sample.txt $sample.yml

### creating sample.yml

In [5]:
%%tcl
info vars

'PROGRAM_NAME argv tcl_version tcl_interactive argv0 sample auto_path errorCode errorInfo auto_index RealScript env tcl_pkgPath LOGFILE tcl_patchLevel SHELLTOOL_LOC RealBin argc tcl_library tcl_platform _tkinter_skip_tk_init'

## yml2tcl

In [6]:
%%tcl
proc yml2tcl::import {sample {level 1}} {
    dict for {key value} [yaml::yaml2dict -file $sample] {
        upvar $level $key $key
        if {[array exists value]} {
            array set $name_array $value
        } else {
            set $key $value
        }
    }
    foreach {key value} $releaseMacro {
        set var_name "releaseMacro\{$key\}"
        upvar $level $var_name $var_name  
        set $var_name $value
    }
}

TclError: can't create procedure "yml2tcl::import": unknown namespace

In [7]:
%%tcl
yml2tcl sample.yml

TclError: invalid command name "yml2tcl"

### get list of variables from yml


In [8]:
%%tcl
proc yml2tcl::getvalue {sample var_name} {
    dict for {key value} [yaml::yaml2dict -file $sample] {
        if {$key == $var_name} {return $value}
    }
    eprint "The variable $var_name was not found"
    return error
}

TclError: can't create procedure "yml2tcl::getvalue": unknown namespace